<a href="https://colab.research.google.com/github/muwongelawrence-tech/colab_projects/blob/main/weed_crop_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import glob
import xml.etree.ElementTree as ET
import pandas as pd
import tensorflow as tf
print(tf.__version__)

2.8.2


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'AI Machine learning and data science projects'
 CERTIFICATES
'classifier model with its lite version'
 classifier-to-detector
'Colab Notebooks'
'Coursera courses'
 customTF2
'Discrete Mathematics and Its Applications, 7th Ed - Kenneth H Rosen.pdf'
'Emerging trends report.gdoc'
'End of Semester 3'
'End of semester 4'
'FASTER_RCNN PAPER.pdf'
'Git Cheat Sheet~.pdf'
'Group 2 indirect communications .gdoc'
'Important files'
'important info'
 INTERNSHIP2021
'ML documents'
'My Drive'
'My files '
'PHY O LEVEL PAST PAPERS'
'programming books'
'Programming materials'
'RECESS CISCO'
'Reference books'
'scanned work'
'semester 5'
'semester 6 (current)'
'web design by deitel.pdf'
 weedDetectorTF2
'Work and Teahing materials'


In [4]:
# clone the tensorflow models on the colab cloud vm
!git clone --q https://github.com/tensorflow/models.git

#navigate to /models/research folder to compile protos
%cd models/research

# Compile protos.
!protoc object_detection/protos/*.proto --python_out=.

# Install TensorFlow Object Detection API.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

/content/models/research
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 10.9 MB 9.4 MB/s 
     |████████████████████████████████| 352 kB 70.8 MB/s 
     |████████████████████████████████| 2.1 MB 58.7 MB/s 
     |████████████████████████████████| 25.0 MB 1.3 MB/s 
     |████████████████████████████████| 67 kB 7.8 MB/s 
     |████████████████████████████████| 116 kB 68.2 MB/s 
     |████████████████████████████████| 511.8 MB 8.8 kB/s 
     |████████████████████████████████|

In [5]:
# testing the model builder
!python object_detection/builders/model_builder_tf2_test.py

Running tests under Python 3.7.14: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
2022-09-16 13:45:43.971204: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
W0916 13:45:44.306618 139951623780224 model_builder.py:1102] Building experimental DeepMAC meta-arch. Some features may be omitted.
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 1.52s
I0916 13:45:44.562109 139951623780224 test_util.py:2459] time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 1.52s
[       OK ] ModelBuilderTF2Test.test_create_center_net_deepmac
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_model0 (customize_head_params=True)
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_center_net_model0 (customize_head_params=True)): 0.51s
I0916 13:45:45.068608 139951623780224 test_u

In [7]:
%cd /mydrive/weedDetectorTF2/data/

# unzip the datasets and their contents so that they are now in /mydrive/customTF2/data/ folder
# !unzip /mydrive/customTF2/images.zip -d .
# !unzip /mydrive/customTF2/annotations.zip -d .


/content/gdrive/My Drive/weedDetectorTF2/data
/content/gdrive/My Drive/weedDetectorTF2/data


In [8]:
#creating two dir for training and testing
!mkdir test_labels train_labels

# lists the files inside 'annotations' in a random order (not really random, by their hash value instead)
# Moves the first 195/972 labels (20% of the labels) to the testing dir: `test_labels`
!ls annotations/* | sort -R | head -195 | xargs -I{} mv {} test_labels/


# Moves the rest of the labels ( 1096 labels ) to the training dir: `train_labels`
!ls annotations/* | xargs -I{} mv {} train_labels/

In [10]:
def xml_to_csv(path):
  classes_names = []
  xml_list = []

  for xml_file in glob.glob(path + '/*.xml'):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    for member in root.findall('object'):
      classes_names.append(member[0].text)
      value = (root.find('filename').text  ,   
               int(root.find('size')[0].text),
               int(root.find('size')[1].text),
               member[0].text,
               int(member[4][0].text),
               int(member[4][1].text),
               int(member[4][2].text),
               int(member[4][3].text))
      xml_list.append(value)
  column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
  xml_df = pd.DataFrame(xml_list, columns=column_name) 
  classes_names = list(set(classes_names))
  classes_names.sort()
  return xml_df, classes_names

for label_path in ['train_labels', 'test_labels']:
  image_path = os.path.join(os.getcwd(), label_path)
  xml_df, classes = xml_to_csv(label_path)
  xml_df.to_csv(f'{label_path}.csv', index=None)
  print(f'Successfully converted {label_path} xml to csv.')

label_map_path = os.path.join("label_map.pbtxt")
pbtxt_content = ""

for i, class_name in enumerate(classes):
    pbtxt_content = (
        pbtxt_content
        + "item {{\n    id: {0}\n    name: '{1}'\n}}\n\n".format(i + 1, class_name)
    )
pbtxt_content = pbtxt_content.strip()
with open(label_map_path, "w") as f:
    f.write(pbtxt_content)
    print('Successfully created label_map.pbtxt ')  

Successfully converted train_labels xml to csv.
Successfully converted test_labels xml to csv.
Successfully created label_map.pbtxt 


In [12]:
#Usage:  
#!python generate_tfrecord.py output.csv output_pb.txt /path/to/images output.tfrecords

#For train.record
!python /mydrive/weedDetectorTF2/generate_tfrecord.py train_labels.csv  label_map.pbtxt images/ train.record

#For test.record
!python /mydrive/weedDetectorTF2/generate_tfrecord.py test_labels.csv  label_map.pbtxt images/ test.record

python3: can't open file '/mydrive/weedDetectorTF2/generate_tfrecord.py': [Errno 2] No such file or directory
python3: can't open file '/mydrive/weedDetectorTF2/generate_tfrecord.py': [Errno 2] No such file or directory
